In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from termcolor import colored
import requests
import io
import time

def download_nse_stocks():
    """
    Download list of all NSE stocks
    """
    try:
        # Download from NSE website (equity securities)
        url = "https://archives.nseindia.com/content/equities/EQUITY_L.csv"
        response = requests.get(url)
        df = pd.read_csv(io.StringIO(response.content.decode('utf-8')))
        return df['SYMBOL'].tolist()
    except Exception as e:
        print(f"Error downloading NSE stocks: {e}")
        return []

def download_bse_stocks():
    """
    Download list of all BSE stocks
    """
    try:
        # Download from BSE website (equity securities)
        url = "https://www.bseindia.com/stock-share-price/stockreach_downloads.aspx"
        response = requests.get(url)
        df = pd.read_csv(io.StringIO(response.content.decode('utf-8')))
        return df['Security Id'].tolist()
    except Exception as e:
        print(f"Error downloading BSE stocks: {e}")
        return []

def calculate_rsi(data, periods=14):
    """
    Calculate RSI (Relative Strength Index)
    """
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=periods).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def get_stock_data(symbol, exchange='NSE', days=200):
    """
    Fetch stock data with proper exchange suffix
    """
    try:
        # Add appropriate suffix based on exchange
        ticker_symbol = f"{symbol}.NS" if exchange == 'NSE' else f"{symbol}.BO"
        stock = yf.Ticker(ticker_symbol)
        
        # Get historical data
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days)
        hist_data = stock.history(start=start_date, end=end_date)
        
        if hist_data.empty:
            return None, None, None
        
        # Calculate RSI
        hist_data['RSI'] = calculate_rsi(hist_data['Close'])
        
        latest_price = hist_data['Close'].iloc[-1]
        latest_rsi = hist_data['RSI'].iloc[-1]
        
        return latest_price, hist_data, latest_rsi
    
    except Exception as e:
        return None, None, None

def analyze_stocks(symbols, exchange, batch_size=100):
    """
    Analyze stocks in batches to handle rate limiting
    """
    results = []
    total_symbols = len(symbols)
    
    for i in range(0, total_symbols, batch_size):
        batch_symbols = symbols[i:i+batch_size]
        print(f"\nProcessing batch {i//batch_size + 1}/{(total_symbols+batch_size-1)//batch_size} for {exchange}...")
        
        for symbol in batch_symbols:
            latest_price, hist_data, latest_rsi = get_stock_data(symbol, exchange)
            
            if latest_price is not None and latest_rsi is not None:
                results.append({
                    'Symbol': symbol,
                    'Exchange': exchange,
                    'Latest_Price': latest_price,
                    'RSI': latest_rsi,
                    'Volume': hist_data['Volume'].iloc[-1] if 'Volume' in hist_data else 0
                })
        
        # Add delay to avoid rate limiting
        time.sleep(1)
    
    return pd.DataFrame(results)

def save_results(df, filename):
    """
    Save results with proper formatting
    """
    df.sort_values('RSI', ascending=False, inplace=True)
    df.to_csv(filename, index=False)
    return df

def main():
    # Create output directory if it doesn't exist
    import os
    if not os.path.exists('stock_analysis'):
        os.makedirs('stock_analysis')
    
    # Get current date for filenames
    current_date = datetime.now().strftime('%Y%m%d')
    
    print("Downloading stock lists...")
    nse_symbols = download_nse_stocks()
    bse_symbols = download_bse_stocks()
    
    print(f"\nFound {len(nse_symbols)} NSE stocks and {len(bse_symbols)} BSE stocks")
    
    # Analyze NSE stocks
    print("\nAnalyzing NSE stocks...")
    nse_results = analyze_stocks(nse_symbols, 'NSE')
    nse_results = save_results(nse_results, f'stock_analysis/nse_rsi_analysis_{current_date}.csv')
    
    # Analyze BSE stocks
    print("\nAnalyzing BSE stocks...")
    bse_results = analyze_stocks(bse_symbols, 'BSE')
    bse_results = save_results(bse_results, f'stock_analysis/bse_rsi_analysis_{current_date}.csv')
    
    # Print summary of stocks with RSI >= 40
    print("\nStocks with RSI >= 40:")
    print("\nNSE Stocks:")
    nse_high_rsi = nse_results[nse_results['RSI'] >= 40].sort_values('RSI', ascending=False)
    for _, row in nse_high_rsi.iterrows():
        print(colored(f"{row['Symbol']}: RSI = {row['RSI']:.2f}, Price = ₹{row['Latest_Price']:.2f}", 
                     'green', attrs=['bold']))
    
    print("\nBSE Stocks:")
    bse_high_rsi = bse_results[bse_results['RSI'] >= 40].sort_values('RSI', ascending=False)
    for _, row in bse_high_rsi.iterrows():
        print(colored(f"{row['Symbol']}: RSI = {row['RSI']:.2f}, Price = ₹{row['Latest_Price']:.2f}", 
                     'green', attrs=['bold']))
    
    # Save combined summary of high RSI stocks
    high_rsi_summary = pd.concat([nse_high_rsi, bse_high_rsi])
    high_rsi_summary.to_csv(f'stock_analysis/high_rsi_stocks_{current_date}.csv', index=False)
    
    print(f"\nAnalysis complete. Results saved in stock_analysis directory.")
    print(f"Total stocks with RSI >= 40: {len(high_rsi_summary)}")

if __name__ == "__main__":
    # Install required packages if not already installed
    # pip install yfinance pandas numpy requests termcolor
    
    main()

Error downloading BSE stocks: 'Security Id'

Found 2087 NSE stocks and 0 BSE stocks

Analyzing NSE stocks...

Processing batch 1/21 for NSE...

Processing batch 2/21 for NSE...

Processing batch 3/21 for NSE...

Processing batch 4/21 for NSE...

Processing batch 5/21 for NSE...

Processing batch 6/21 for NSE...

Processing batch 7/21 for NSE...

Processing batch 8/21 for NSE...

Processing batch 9/21 for NSE...

Processing batch 10/21 for NSE...


$KALYANI.NS: possibly delisted; no price data found  (1d 2024-06-20 13:56:28.222436 -> 2025-01-06 13:56:28.222436)



Processing batch 11/21 for NSE...

Processing batch 12/21 for NSE...


MOKSH-RE.NS: Period '1mo' is invalid, must be one of ['1d', '5d']



Processing batch 13/21 for NSE...

Processing batch 14/21 for NSE...

Processing batch 15/21 for NSE...

Processing batch 16/21 for NSE...

Processing batch 17/21 for NSE...

Processing batch 18/21 for NSE...

Processing batch 19/21 for NSE...

Processing batch 20/21 for NSE...

Processing batch 21/21 for NSE...

Analyzing BSE stocks...


KeyError: 'RSI'

In [2]:
pip install yfinance pandas numpy requests